In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [25]:
CBWs         = [    2,    4,    6]
PBWs         = [   10,   16,   24]
CWNDs        = [   16,   64,  256,  512, 1024]
TTIME        = 5
NUMEXP       = '1-250' # whatever the fuck this means?
TPTALGOs     = ['TCPreno','TCPcubic'] #,'UDP']
ATK_PARA     = list(range(1, 252, 5))
hostnames    = ['atkr'] + [f'h{i}' for i in range(8)]
iperf_header = ['timestamp','src_addr','src_port','dest_addr','dest_port','transferID','interval','transferred_bytes','bits_per_second']

indices      = [('CBW', CBWs), ('PBW', PBWs), ('CWND', CWNDs), ('TPTALGO', TPTALGOs), ('APC', ATK_PARA)]

basedir   = f'./NUMEXP-1-250_TTIME-5_STEP-5'

In [26]:
dfall = []
for CBW in CBWs:
    for PBW in PBWs:
        for CWND in CWNDs: 
            for TPTALGO in TPTALGOs:
                for APC in ATK_PARA:
                    for HOST in hostnames:
                        dfi = pd.read_csv(f'{basedir}/CBW-{CBW}/PBW-{PBW}/CWND-{CWND}K/{TPTALGO}/{APC}conn/iperf_{HOST}.csv', names=iperf_header)
                        dfi['host']    = HOST
                        dfi['APC']     = APC
                        dfi['TPTALGO'] = TPTALGO
                        dfi['CWND']    = CWND
                        dfi['PBW']     = PBW
                        dfi['CBW']     = CBW
                        dfall.append(dfi)
                        
dfbigboi = pd.concat(dfall)
dfbigboi

,timestamp,src_addr,src_port,dest_addr,dest_port,transferID,interval,transferred_bytes,bits_per_second,host,APC,TPTALGO,CWND,PBW,CBW
0,20211123172644,10.0.0.1,60698,10.0.0.10,5001,4,0.0-1.0,60816,486528,atkr,6,TCPreno,16,10,2
1,20211123172644,10.0.0.1,60708,10.0.0.10,5001,8,0.0-1.0,59368,474944,atkr,6,TCPreno,16,10,2
2,20211123172644,10.0.0.1,60696,10.0.0.10,5001,3,0.0-1.0,59368,474944,atkr,6,TCPreno,16,10,2
3,20211123172644,10.0.0.1,60706,10.0.0.10,5001,7,0.0-1.0,53576,428608,atkr,6,TCPreno,16,10,2
4,20211123172644,10.0.0.1,60702,10.0.0.10,5001,5,0.0-1.0,59368,474944,atkr,6,TCPreno,16,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,20211124061050,10.0.0.9,48774,10.0.0.10,5001,3,1.0-2.0,262144,2097152,h7,251,TCPcubic,1024,24,6
2,20211124061051,10.0.0.9,48774,10.0.0.10,5001,3,2.0-3.0,262144,2097152,h7,251,TCPcubic,1024,24,6
3,20211124061052,10.0.0.9,48774,10.0.0.10,5001,3,3.0-4.0,262144,2097152,h7,251,TCPcubic,1024,24,6
4,20211124061053,10.0.0.9,48774,10.0.0.10,5001,3,4.0-5.0,262144,2097152,h7,251,TCPcubic,1024,24,6


In [28]:
del dfall

NameError: name 'dfall' is not defined

In [ ]:
SCALAR_FIG_SZ = 5

for indeX in indices:
    for indeY in indices:
        if indices.index(indeX) == indices.index(indeY): continue
        AX_R_NAME, AX_R_VALS = indeX
        AX_C_NAME, AX_C_VALS = indeY
        AX_R_LEN , AX_C_LEN  = len(AX_R_VALS), len(AX_C_VALS)
        print(f'{AX_R_LEN} x {AX_C_LEN}\t', AX_R_NAME, '\tvs.  ', AX_C_NAME)
        
        fig, axs = plt.subplots(AX_R_LEN, AX_C_LEN, figsize=(AX_C_LEN*SCALAR_FIG_SZ, AX_R_LEN*SCALAR_FIG_SZ))
        
        maxY = 0
        for R_VAL in AX_R_VALS:
            df0 = dfbigboi[dfbigboi[AX_R_NAME] == R_VAL]
            for C_VAL in AX_C_VALS:
                df1 = df0[df0[AX_C_NAME] == C_VAL]
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                
                net_data = [0] * len(range(TTIME))
                for HOST in hostnames:
                    df2 = df1[df1['host'] == HOST]
                    host_data = []
                    for t in range(TTIME):
                        tint = f'{t:.1f}-{t+1:.1f}'
                        df3 = df2[df2['interval'] == tint]
#                         print(HOST, df3.shape)
                        if HOST == 'atkr' and df3.shape[0] > 1:
                            df3 = df3[df3['transferID'] == -1]
#                         print(HOST, df3.shape)
                        host_total = df3.sum()['transferred_bytes'] // df3.shape[0]
#                         if(host_total != host_total):
#                             print(df3)
                        host_data.append(host_total)
                        net_data[t] += host_total
                    maxY = max(maxY, max(host_data), max(net_data))
#                     print(HOST, host_data)
                    axi.plot(range(TTIME), host_data, label=HOST)
                axi.plot(range(TTIME), net_data, label='network')
                axi.legend()
                axi.set_xlabel('time')
                axi.set_ylabel('transferred_bytes')
                axi.set_title(f'{AX_R_NAME} {R_VAL} vs. {AX_C_NAME} {C_VAL}')
                
        for R_VAL in AX_R_VALS:
            for C_VAL in AX_C_VALS:
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                axi.set_ylim(bottom=0, top=maxY)
                
#         break
#     break

        fig.savefig(f'./VIZTIME/VIZTIME_{AX_R_NAME}-{AX_C_NAME}.png', facecolor="w")

In [34]:
SCALAR_FIG_SZ = 5

missing_data = 0
t_data = [t+0.5 for t in range(TTIME)]

maxY = 242580000 

for CBW in CBWs:
    df0 = dfbigboi[dfbigboi['CBW'] == CBW]
    for PBW in PBWs:
        df1 = df0[df0['PBW'] == PBW]
        for CWND in CWNDs: 
            df2 = df1[df1['CWND'] == CWND]
            for TPTALGO in TPTALGOs:
                df3 = df2[df2['TPTALGO'] == TPTALGO]
                print(CBW, PBW, CWND, TPTALGO)
                for APC in ATK_PARA:
                    df4 = df3[df3['APC'] == APC]
#                     print(CBW, PBW, CWND, TPTALGO, APC)
                    fig, axs = plt.subplots(1, 1, figsize=(SCALAR_FIG_SZ, SCALAR_FIG_SZ))
                    axs.set_ylim(bottom=-1, top=maxY)
                    net_data = [0] * len(t_data)
                    for HOST in hostnames:
                        host_data = []
                        df5 = df4[df4['host'] == HOST]
                        for t in range(TTIME):
                            tint = f'{t:.1f}-{t+1:.1f}'
                            df6 = df5[df5['interval'] == tint]
                            if df6.shape[0] > 1:
                                df6 = df6[df6['transferID'] == -1]
                            try:
                                host_data.append(df6['transferred_bytes'].values[0])
                            except:
                                host_data.append(-1)
                                missing_data += 1
                            net_data[t] += host_data[-1]
#                             maxY = max(maxY, host_data[-1], net_data[t])
                        axs.plot(t_data, host_data, label=HOST)
                    axs.plot(t_data, net_data, label='network')
                    axs.legend()
                    axs.set_xlabel('time')
                    axs.set_ylabel('transferred_bytes')
                    axs.set_title(f'CBW{CBW} PBW{PBW} {CWND}K {TPTALGO} APC{APC}')
                    fig.savefig(f'./VIZLITTLESHITS/APC{APC:03d}-CBW{CBW}-PBW{PBW}-{CWND}K-{TPTALGO}.png', facecolor="w")
                    plt.close()
                    
print('missing_data:', missing_data)
print('maxY', maxY)

2 10 16 TCPreno
2 10 16 TCPcubic
2 10 64 TCPreno
2 10 64 TCPcubic
2 10 256 TCPreno
2 10 256 TCPcubic
2 10 512 TCPreno
2 10 512 TCPcubic
2 10 1024 TCPreno
2 10 1024 TCPcubic
2 16 16 TCPreno
2 16 16 TCPcubic
2 16 64 TCPreno
2 16 64 TCPcubic
2 16 256 TCPreno
2 16 256 TCPcubic
2 16 512 TCPreno
2 16 512 TCPcubic
2 16 1024 TCPreno
2 16 1024 TCPcubic
2 24 16 TCPreno
2 24 16 TCPcubic
2 24 64 TCPreno
2 24 64 TCPcubic
2 24 256 TCPreno
2 24 256 TCPcubic
2 24 512 TCPreno
2 24 512 TCPcubic
2 24 1024 TCPreno
2 24 1024 TCPcubic
4 10 16 TCPreno
4 10 16 TCPcubic
4 10 64 TCPreno
4 10 64 TCPcubic
4 10 256 TCPreno
4 10 256 TCPcubic
4 10 512 TCPreno
4 10 512 TCPcubic
4 10 1024 TCPreno
4 10 1024 TCPcubic
4 16 16 TCPreno
4 16 16 TCPcubic
4 16 64 TCPreno
4 16 64 TCPcubic
4 16 256 TCPreno
4 16 256 TCPcubic
4 16 512 TCPreno
4 16 512 TCPcubic
4 16 1024 TCPreno
4 16 1024 TCPcubic
4 24 16 TCPreno
4 24 16 TCPcubic
4 24 64 TCPreno
4 24 64 TCPcubic
4 24 256 TCPreno
4 24 256 TCPcubic
4 24 512 TCPreno
4 24 512 TCPcubic

In [29]:
[t+0.5 for t in range(TTIME)]

[0.5, 1.5, 2.5, 3.5, 4.5]